<a href="https://colab.research.google.com/github/Yujin726/0624/blob/main/07_24_roboflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Cp3duFiPkojcl1xV3RLS")
project = rf.workspace("0721yolov8").project("0723_kyj-1twmf")
dataset = project.version(2).download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to 0723_kyj-2 in yolov11:: 100%|██████████| 378/378 [00:00<00:00, 6585.45it/s]


In [ ]:
# 1. 필요한 라이브러리 설치
!pip install yt-dlp roboflow ultralytics opencv-python

import cv2
import os
from roboflow import Roboflow
import numpy as np
from ultralytics import YOLO
import yt_dlp   # 차선 감지 모델을 위한 YouTube 영상 처리
# Roboflow에서 이미 훈련된 모델을 불러와서 사용
#영상에서 프레임을 추출하여 **추론(inference)**만 수행
import cv2
import os
from roboflow import Roboflow
import yt_dlp

def loa

def download_driving_video(url, output_path="./"):
    """
    드라이빙 영상 다운로드
    """
    ydl_opts = {
        'format': 'best[height<=720]',
        'outtmpl': os.path.join(output_path, 'driving_video.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])
            print("✅ 드라이빙 영상 다운로드 완료!")
            return True
        except Exception as e:
            print(f"❌ 다운로드 실패: {e}")
            return False

def detect_lanes_in_video(video_path, model, output_path="lane_detection_result.mp4"):
    """
    영상에서 차선 감지
    """
    cap = cv2.VideoCapture(video_path)

    # 영상 정보
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"🎬 영상 정보: {width}x{height}, {fps}fps, {total_frames}프레임")

    # 출력 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    lane_detections = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # 매 3프레임마다 차선 감지 (더 자주)
        if frame_count % 2 == 0:
            try:
                temp_img_path = "temp_frame.jpg"
                cv2.imwrite(temp_img_path, frame)

                # 차선 감지 (낮은 신뢰도)
                prediction = model.predict(temp_img_path, confidence=30, overlap=50)
                predictions = prediction.json()['predictions']

                frame_lanes = len(predictions)
                lane_detections += frame_lanes

                if frame_lanes > 0:
                    print(f"🛣️ 프레임 {frame_count}: {frame_lanes}개 차선 감지")

                # 차선 그리기
                for lane in predictions:
                    x1 = int(lane['x'] - lane['width']/2)
                    y1 = int(lane['y'] - lane['height']/2)
                    x2 = int(lane['x'] + lane['width']/2)
                    y2 = int(lane['y'] + lane['height']/2)

                    # 차선은 보라색으로 표시
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)

                    # 라벨
                    label = f"Lane: {lane['confidence']:.2f}"
                    cv2.rectangle(frame, (x1, y1-30), (x1+150, y1), (255, 0, 255), -1)
                    cv2.putText(frame, label, (x1+5, y1-8),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

                if os.path.exists(temp_img_path):
                    os.remove(temp_img_path)

            except Exception as e:
                print(f"❌ 프레임 {frame_count} 처리 오류: {e}")

        out.write(frame)

        # 진행상황
        if frame_count % 150 == 0:
            progress = (frame_count / total_frames) * 100
            print(f"📊 진행률: {progress:.1f}% (총 차선 감지: {lane_detections}개)")

    cap.release()
    out.release()

    print(f"✅ 완료! 총 {lane_detections}개 차선 감지")
    print(f"🎥 결과: {output_path}")

def main_lane_detection():
    """
    차선 감지 메인 함수
    """
    # 추천 YouTube URL들 (드라이빙 영상)
    driving_urls = [
        "https://www.youtube.com/watch?v=AxLmroTo3rQ",  # 기존 URL
        # 더 나은 드라이빙 영상이 있다면 교체하세요
    ]

    youtube_url = driving_urls[0]

    print("🚗 드라이빙 영상 다운로드 시작...")
    if download_driving_video(youtube_url):

        # 다운로드된 파일 찾기
        video_files = [f for f in os.listdir('.') if f.startswith('driving_video')]
        if video_files:
            video_path = video_files[0]
            print(f"📁 영상 파일: {video_path}")

            print("🛣️ 차선 감지 모델 로드 중...")
            try:
                model = load_lane_detection_model()

                print("🔍 차선 감지 시작...")
                detect_lanes_in_video(video_path, model, "lane_detection_result.mp4")

                print("🎉 차선 감지 완료!")
                print("📺 결과 영상: lane_detection_result.mp4")

            except Exception as e:
                print(f"❌ 모델 로드 실패: {e}")
                print("🔑 API 키를 확인해주세요!")
        else:
            print("❌ 다운로드된 영상 파일을 찾을 수 없습니다.")

def test_with_webcam_url():
    """
    Roboflow Visualize 페이지의 'Paste YouTube or Image URL' 기능 사용
    """
    print("🌐 웹 인터페이스 테스트:")
    print("1. Roboflow Visualize 페이지에서")
    print("2. 'Paste YouTube or Image URL' 입력창에")
    print("3. YouTube URL 붙여넣기")
    print("4. 차선이 잘 감지되는지 확인")

# 실행
if __name__ == "__main__":
    print("🛣️ 차선 감지 모드로 변경!")
    print("=" * 50)
    print("💡 이 모델은 차선(lane)을 감지하는 모델입니다.")
    print("📹 드라이빙 영상이나 도로 영상에서 가장 잘 작동합니다.")
    print()
    print("🚀 실행 방법:")
    print("1. API 키 입력")
    print("2. main_lane_detection() 실행")
    print("3. 또는 Roboflow 웹에서 test_with_webcam_url() 방법 시도")
    print("=" * 50)
    main_lane_detection()